In [ ]:
!pip install liboidcagent requests xmltodict pandas boto3

In [ ]:
!eval `oidc-keychain` > /dev/null && oidc-token dodas --time=3600 > /tmp/token
with open('/tmp/token') as f:
    token = f.readlines()[0].split("\n")[0]

In [ ]:
import requests
import xmltodict
r = requests.post("https://minio.cloud.infn.it",
                  data={
                      'Action':
                      "AssumeRoleWithWebIdentity",
                      'Version': "2011-06-15",
                      'WebIdentityToken': token,
                      'DurationSeconds': 9000
                  },
                  verify=True)

tree = xmltodict.parse(r.content)

credentials = dict(tree['AssumeRoleWithWebIdentityResponse']
                    ['AssumeRoleWithWebIdentityResult']['Credentials'])

In [ ]:
# Read CSV
import pandas as pd
import boto3
s3 = boto3.client('s3',endpoint_url="https://minio.cloud.infn.it",aws_access_key_id=credentials["AccessKeyId"],
                  aws_secret_access_key=credentials["SecretAccessKey"],
                  aws_session_token=credentials["SessionToken"],
                  config=boto3.session.Config(signature_version='s3v4'),
                 verify=True)

read_file = s3.get_object(Bucket='scratch', Key='verlato/dpc-covid19-ita-regioni.csv')
df = pd.read_csv(read_file['Body'],sep=',')
df